# WeChat Anti-Recall

This script is in a notebook for ease of configuration.

In [1]:
# dependencies

import os
import bs4
import itchat
import collections

import itchat.content as ct

from io import StringIO
import contextlib
import traceback

In [2]:
# configurations

class Config:
    
    # whitelist of all group chat names to enable anti-recall in
    GROUPS = [ '23 CMU 校友', 'test' ]
    
    # msg id of a "recall" message
    RECALL_ID = 10002
    
    # expire time of a stored message (default: 2 minutes)
    EXPIRE = 120
    
    # path of image folder
    IMG_PATH = 'image_cache'

In [3]:
# message handlers

class BaseMessage:
    
    def __init__(self, data, time):
        self.data = data
        self.time = time
    
    def on_recall(self, msg): raise NotImplementedError
    def on_delete(self): raise NotImplementedError
    
class TextMessage(BaseMessage):
        
    def on_recall(self, msg):
        msg.user.send(f'「撤回提示」@{msg.actualNickName}\u2005撤回了一条信息，内容为：{self.data}')
        
    def on_delete(self):
        pass
    
class PictureMessage(BaseMessage):

    def on_recall(self, msg):
        msg.user.send(f'「撤回提示」@{msg.actualNickName}\u2005撤回了一张图片：')
        msg.user.send_image(self.data)
        
    def on_delete(self):
        os.remove(self.data)

In [4]:
# session

history = collections.OrderedDict()

# itchat is single-threaded so this is fine
def clean_up(curr_time):
    remove = []
    for k, v in history.items():
        if curr_time - v.time < Config.EXPIRE:
            break
            
        remove.append(k)
    
    for k in remove:
        history[k].on_delete()
        del history[k]

In [5]:
# event subscribers

dedup_hack = set()

def get_exec(code):
    s = StringIO()
    with contextlib.redirect_stdout(s):
        try:
            exec(code, {})
        except:
            print(traceback.format_exc())
    return s.getvalue()

# itchat-uos' `itchat.content.*` is bugged and actual only accepts lower-case types,
# thus the manual entering of types
@itchat.msg_register(['Picture', 'Video'], isGroupChat=True)
def img_msg(msg):
    if msg.user.nickName not in Config.GROUPS or msg.msgId in history:
        return

    clean_up(msg.createTime)
    path = os.path.join(Config.IMG_PATH, msg.fileName)
    msg.download(path)
    history[msg.msgId] = PictureMessage(path, msg.createTime)
    
@itchat.msg_register(ct.TEXT, isGroupChat=True)
def group_text_msg(msg):
    if msg.user.nickName not in Config.GROUPS or msg.msgId in dedup_hack:
        return
    
    dedup_hack.add(msg.msgId)
    
    if msg.content.startswith('python:'):
        code = '\n'.join(msg.content.split('\n')[1 :])
        res = get_exec(code)
        msg.user.send(res)
    else:
        if msg.msgId in history:
            return

        clean_up(msg.createTime)
        history[msg.msgId] = TextMessage(msg.content, msg.createTime)

@itchat.msg_register(ct.NOTE, isGroupChat=True)
def note_msg(msg):
    if msg.user.nickName not in Config.GROUPS:
        return
    
    if msg.msgType == Config.RECALL_ID:
        obj = bs4.BeautifulSoup(msg.content, 'lxml')
        msg_id = obj.find('msgid').text
        if msg_id in history:
            original = history[msg_id]
            original.on_recall(msg)
            original.on_delete()
            del history[msg_id]

In [6]:
# launch itchat

if os.path.exists(Config.IMG_PATH):
    for f in os.listdir(Config.IMG_PATH):
        if f[0] != '.':
            os.remove(os.path.join(Config.IMG_PATH, f))
else:
    os.makedirs(Config.IMG_PATH)

history.clear()
    
itchat.auto_login(hotReload=True)
itchat.run()

Getting uuid of QR code.
Please scan the QR code to log in.
Please press confirm on your phone.
Loading the contact, this may take a little while.
Login successfully as 晓马
Start auto replying.


Bye~
LOG OUT!


In [10]:
try:
    exec('print(1/0)')
except:
    pass